In [4]:
import logging
from itertools import combinations

import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import random
from icecream import ic

In [5]:
class Problem:
    _graph: nx.Graph
    _alpha: float
    _beta: float

    def __init__(
        self,
        num_cities: int,
        *,
        alpha: float = 1.0,
        beta: float = 1.0,
        density: float = 0.5,
        seed: int = 42,
    ):
        rng = np.random.default_rng(seed)
        self._alpha = alpha
        self._beta = beta
        cities = rng.random(size=(num_cities, 2))
        cities[0, 0] = cities[0, 1] = 0.5

        self._graph = nx.Graph()
        self._graph.add_node(0, pos=(cities[0, 0], cities[0, 1]), gold=0)
        for c in range(1, num_cities):
            self._graph.add_node(c, pos=(cities[c, 0], cities[c, 1]), gold=(1 + 999 * rng.random()))

        tmp = cities[:, np.newaxis, :] - cities[np.newaxis, :, :]
        d = np.sqrt(np.sum(np.square(tmp), axis=-1))
        for c1, c2 in combinations(range(num_cities), 2):
            if rng.random() < density or c2 == c1 + 1:
                self._graph.add_edge(c1, c2, dist=d[c1, c2])

        assert nx.is_connected(self._graph)

    @property
    def graph(self) -> nx.Graph:
        return nx.Graph(self._graph)

    @property
    def dist_dict(self):
        return {(u, v): data['dist'] for u, v, data in self._graph.edges(data=True)}

    @property
    def gold_dict(self):
        return {n: data['gold'] for n, data in self._graph.nodes(data=True)}

    def cost(self, path, weight):
        dist = nx.path_weight(self._graph, path, weight='dist')
        return dist + (self._alpha * dist * weight) ** self._beta

    def baseline(self):
        total_cost = 0
        for dest, path in nx.single_source_dijkstra_path(
            self._graph, source=0, weight='dist'
        ).items():
            cost = 0
            for c1, c2 in zip(path, path[1:]):
                cost += self.cost([c1, c2], 0)
                cost += self.cost([c1, c2], self._graph.nodes[dest]['gold'])
            logging.debug(
                f"dummy_solution: go to {dest} ({' > '.join(str(n) for n in path)} ({cost})"
            )
            total_cost += cost
        return total_cost

    def plot(self):
        plt.figure(figsize=(10, 10))
        pos = nx.get_node_attributes(self._graph, 'pos')
        size = [100] + [self._graph.nodes[n]['gold'] for n in range(1, len(self._graph))]
        color = ['red'] + ['lightblue'] * (len(self._graph) - 1)
        return nx.draw(self._graph, pos, with_labels=True, node_color=color, node_size=size)

In [6]:
logging.getLogger().setLevel(logging.WARNING)

ic(Problem(100, density=0.2, alpha=1, beta=1).baseline())
ic(Problem(100, density=0.2, alpha=2, beta=1).baseline())
ic(Problem(100, density=0.2, alpha=1, beta=2).baseline())
ic(Problem(100, density=1, alpha=1, beta=1).baseline())
ic(Problem(100, density=1, alpha=2, beta=1).baseline())
ic(Problem(100, density=1, alpha=1, beta=2).baseline())
ic(Problem(1_000, density=0.2, alpha=1, beta=1).baseline())
ic(Problem(1_000, density=0.2, alpha=2, beta=1).baseline())
ic(Problem(1_000, density=0.2, alpha=1, beta=2).baseline())
ic(Problem(1_000, density=1, alpha=1, beta=1).baseline())
ic(Problem(1_000, density=1, alpha=2, beta=1).baseline())
ic(Problem(1_000, density=1, alpha=1, beta=2).baseline())

None

ic| Problem(100, density=0.2, alpha=1, beta=1).baseline(): np.float64(25266.40561851072)
ic| Problem(100, density=0.2, alpha=2, beta=1).baseline(): np.float64(50425.30961817918)
ic| Problem(100, density=0.2, alpha=1, beta=2).baseline(): np.float64(5334401.927002504)
ic| Problem(100, density=1, alpha=1, beta=1).baseline(): np.float64(18266.18579582672)
ic| Problem(100, density=1, alpha=2, beta=1).baseline(): np.float64(36457.918462372065)
ic| Problem(100, density=1, alpha=1, beta=2).baseline(): np.float64(5404978.08899582)
ic| Problem(1_000, density=0.2, alpha=1, beta=1).baseline(): np.float64(195402.95810394012)
ic| Problem(1_000, density=0.2, alpha=2, beta=1).baseline(): np.float64(390028.72126288974)
ic| Problem(1_000, density=0.2, alpha=1, beta=2).baseline(): np.float64(37545927.70213464)
ic| Problem(1_000, density=1, alpha=1, beta=1).baseline(): np.float64(192936.23377726765)
ic| Problem(1_000, density=1, alpha=2, beta=1).baseline(): np.float64(385105.64149576554)
ic| Problem(1_000

In [57]:
def calculate_full_path_cost_final(problem_instance, path, trip_counts=None):
    """
    Calcola il costo totale del percorso nel formato (nodo, is_active).
    
    PARAMETRI:
    - path: lista di tuple (nodo, is_active)
    - trip_counts: lista opzionale di interi che rappresenta quante volte viene svolto 
                   ogni circuito (viaggio da 0 a 0). Se None, assume 1 per tutti.
    
    LOGICA:
    - Identifica segmenti andata (0 → dest) e ritorno (dest → 0)
    - Per segmento andata: usa weight=0 (nessun oro ancora caricato)
    - Raccoglie oro quando visita nodo con flag True
    - Se il circuito viene svolto n volte, l'oro raccolto = oro_originale / n
    - Per segmento ritorno: usa weight=oro_accumulato
    - Resetta oro quando torna a 0
    
    Restituisce: (infeasible_count, total_cost, infeasible_nodes)
    """
    
    total_cost = 0.0
    current_weight = 0.0
    infeasible_nodes = []
    
    dist_dict = problem_instance.dist_dict
    gold_dict = problem_instance.gold_dict
    
    # Se trip_counts non è fornito, assume 1 per ogni circuito
    if trip_counts is None:
        trip_counts = [1] * count_trips(path)
    
    # Traccia quale circuito siamo
    trip_index = 0
    current_trip_count = trip_counts[trip_index] if trip_index < len(trip_counts) else 1
    
    for i in range(len(path) - 1):
        u = path[i][0]
        v = path[i + 1][0]
        u_flag = path[i][1]  # is_active del nodo corrente
        
        # Se il nodo corrente ha flag True e non è 0, accumula oro
        # Diviso per il numero di volte che questo circuito viene svolto
        if u != 0 and u_flag == True:
            gold_at_u = gold_dict.get(u, 0)
            current_weight += gold_at_u / current_trip_count
        
        # Se il prossimo nodo è 0, siamo alla fine di un circuito
        if v == 0:
            # Finisce il circuito: incrementa l'indice del circuito
            trip_index += 1
            current_trip_count = trip_counts[trip_index] if trip_index < len(trip_counts) else 1
        
        # Resetta il peso quando torniamo a 0
        if u == 0 and current_weight > 0:
            current_weight = 0.0
        
        # Calcola il costo dell'arco u → v
        dist_uv = dist_dict.get((min(u, v), max(u, v)))
        if dist_uv is None:
            # L'arco non esiste: nodo infeasible
            dist_uv = 1.0  # Penalità di default
        # Applica la formula cost() = dist + (alpha * dist * weight)^beta
        cost_segment = dist_uv + (problem_instance._alpha * dist_uv * current_weight) ** problem_instance._beta
        total_cost += cost_segment
    
    return (len(infeasible_nodes), total_cost, infeasible_nodes)


def count_trips(path):
    """
    Conta il numero di circuiti (viaggio da 0 a 0) in un percorso.
    """
    count = 0
    for node, _ in path:
        if node == 0:
            count += 1
    return count


def get_trip_boundaries(path):
    """
    Restituisce una lista di tuple (start_idx, end_idx) che delimita ogni circuito.
    Esempio: Se il path ha 2 circuiti, restituisce [(0, idx1), (idx1, idx2)]
    """
    boundaries = []
    trip_start = 0
    
    for i, (node, _) in enumerate(path):
        if node == 0 and i > 0:
            boundaries.append((trip_start, i))
            trip_start = i
    
    # Aggiungi l'ultimo circuito (che termina all'ultimo elemento)
    if trip_start < len(path):
        boundaries.append((trip_start, len(path) - 1))
    
    return boundaries

In [8]:
def neighborhood_greedy_strategy_dijistra(problem_instance):
   
   
    graph = problem_instance.graph
    node_to_path =dict(nx.single_source_dijkstra_path(graph, source=0, weight='dist'))

    # Crea path_list: lista dei percorsi di ogni nodo ordinati per nodo ID
    path_list = [node_to_path[node] for node in sorted(node_to_path.keys())]
    

    return path_list


In [9]:
def choice_a_path(path_list,seed=42):
    full_path=[]
    rng=random.Random(seed)
    ungolden_nodes=[i for i in range(1, len(path_list))]
    while ungolden_nodes:
        
        node=rng.choice(ungolden_nodes)
        
        path_for_node=list(path_list[node].copy())
        for p in path_for_node[:-1]:
            full_path.append((p, False))
        path_for_node.reverse()
        

        for  p in (path_for_node[:-1]):

            if p in ungolden_nodes:
                full_path.append((p, True))
                ungolden_nodes.remove(p)
                
            else:
                full_path.append((p,False))
            
    full_path.append((0,False))
    return(full_path)


In [10]:
def find_node_with_flag_true(path, wanted_node):

    for i in range(len(path)):
        if path[i][1] and path[i][0] == wanted_node:
            return i
    return -1

In [11]:
def founding_start_and_end_index(path, start_index):
    end_index=start_index+1
    while path[start_index][0]!=0:
        start_index-=1
    while path[end_index][0]!=0:
        end_index+=1

   
    return start_index, end_index

In [12]:
def find_True_flags(path_segment):
    gold_elements=[]
    for node, flag in path_segment:
        if flag:
            gold_elements.append(node)

    return gold_elements

In [13]:
def remove_more_gold_nodes(parent_path, gold_nodes, start_index_2, end_index_2):
    new_path=parent_path.copy()
    index=find_node_with_flag_true(new_path, gold_nodes)
    
    start_index, end_index = founding_start_and_end_index(new_path, index)
    list_of_gold_nodes_in_segment = find_True_flags(new_path[start_index:end_index])
    if len(list_of_gold_nodes_in_segment)==1:
        new_path=new_path[:start_index]+new_path[end_index:]
        delta=end_index-start_index
        start_index_2=start_index_2-delta if start_index_2 > start_index else start_index_2
        end_index_2=end_index_2-delta if end_index_2 > end_index else end_index_2
        return new_path, start_index_2, end_index_2
    new_path[index]=(new_path[index][0], False)

    return new_path, start_index_2, end_index_2

In [14]:
def insert_more_gold_nodes(gold_nodes, path_list):
    full_path=[]
    seed=random.randint(0,1000)
    rng=random.Random(seed)
    ungolden_nodes=gold_nodes.copy()
    while ungolden_nodes:
        
        node=rng.choice(ungolden_nodes)
        
        path_for_node=list(path_list[node].copy())
        for p in path_for_node[:-1]:
            full_path.append((p, False))
        path_for_node.reverse()
        

        for  p in (path_for_node[:-1]):

            if p in ungolden_nodes:
                full_path.append((p, True))
                ungolden_nodes.remove(p)
                
            else:
                full_path.append((p,False))
            
    full_path.append((0,False))

    return full_path

In [15]:
def crossover_zero_paths(parent1, parent2, possible_paths):
    """Esegue croindssover tra due percorsi, restituendo un nuovo percorso figlio."""
    # Trova un punto di crossover casuale
    start_index_1 = random.randint(1, len(parent1) - 2)
    gold_element=0
    index_gold_element_1=-1
    index_gold_element_2=-1
    new_path=[]
    possible_remove=[]
    start_index_1,end_index_1=founding_start_and_end_index(parent1, start_index_1)

    for i in range(start_index_1,end_index_1):
        if parent1[i][1]==True:
            gold_element=parent1[i][0]
            index_gold_element_1=i
            break

    list_gold_1=find_True_flags(parent1[index_gold_element_1+1:end_index_1])
 
    index_gold_element_2=find_node_with_flag_true(parent2, gold_element)
    


    try:
        start_index_2, end_index_2=founding_start_and_end_index(parent2, index_gold_element_2)
    except IndexError:
        raise ValueError("Path ha perso un elemento gold")
    

    list_gold_2=find_True_flags(parent2[start_index_2:end_index_2])

    if len(list_gold_1)>0:
        for node in list_gold_1:
            if node not in list_gold_2:
                parent2, start_index_2, end_index_2=remove_more_gold_nodes(parent2, node, start_index_2, end_index_2)
                

  
    
    list_gold_2.remove(gold_element)
    list_2_not_in_1=[node for node in list_gold_2 if node not in list_gold_1]

   
    if len(list_2_not_in_1)>0:
        new_path=insert_more_gold_nodes(list_2_not_in_1, possible_paths)
    new_individual=parent2[:start_index_2]+ parent1[start_index_1:end_index_1]+ parent2[end_index_2:]+ new_path[1:]



    
    



    return new_individual

In [33]:
def mutation_neighbor_of_next(path, problem_instance, path_list, percentual=0.5):

    new_path = path.copy()
    graph = problem_instance.graph
    insertion_flag=False
    replaced_flag=False
    node_replaced=0


    random_index = random.randint(1, len(new_path) - 2)
    #trovo il settore che sto andando a modificare
    start_index,end_index=founding_start_and_end_index(new_path, random_index)
    for i in range(start_index,end_index):
        if path[i][1]==True:
            first_true_idx=i
            break
    
    
    #salvo il settore per un confronto parziale
    starting_gene=path[start_index:end_index+1]




    if first_true_idx+1 == end_index:
        insertion_flag=True
        idx_to_mutate=first_true_idx
    else:
        candidates_indices = list(range(first_true_idx + 1, end_index))
        idx_to_mutate = random.choice(candidates_indices)
        
        if new_path[idx_to_mutate][1]:
            replaced_flag=True
            node_replaced=new_path[idx_to_mutate][0]
      
    if random.random()< percentual:
        insertion_flag=True        
    
    next_node = new_path[idx_to_mutate + 1][0]

    #posso implementarlo passando tutti i neighboor insieme    
    neighbors = list(graph.neighbors(next_node))
    if 0 in neighbors:
        neighbors.remove(0)
    if new_path[idx_to_mutate][0] in neighbors:
        neighbors.remove(new_path[idx_to_mutate][0])
    if neighbors==[]:
        return path
    new_node = random.choice(neighbors)


    


    if insertion_flag:
        x=nx.shortest_path(graph, source=new_path[idx_to_mutate][0], target=new_node, weight='dist')
        link_path=[]
        for i in range(1, len(x)-1):
            link_path.append((x[i], False))

        new_path=new_path[:idx_to_mutate + 1]+link_path+[(new_node, True)]+new_path[idx_to_mutate + 1:]
    else:
        x=nx.shortest_path(graph, source=new_path[idx_to_mutate-1][0], target=new_node, weight='dist')
        link_path=[]
        for i in range(1, len(x)-1):
            link_path.append((x[i], False))
        new_path=new_path[:idx_to_mutate]+link_path+[(new_node, True)]+new_path[idx_to_mutate + 1:]

    # Cerco dove il nodo era attivo e lo disattivo
    new_path,_,_=remove_more_gold_nodes(new_path,new_node,0,0)
    #se il nodo che ho tolto era attivo lo inserisco nuovamente come nodo da qualche parte
    path_to_append=[]
    if replaced_flag:
        path_to_append=insert_more_gold_nodes([node_replaced], path_list)


        new_path=new_path+path_to_append[1:]


    new_start_index,new_end_index=founding_start_and_end_index(new_path, random_index)

    new_gene=path[new_start_index:new_end_index+1]
    new_gene=new_gene+path_to_append[1:]

    # Calcola correttamente la delta estraendo gli elementi dalla tupla
    new_gene_infeas, new_gene_cost, new_gene_infeas_nodes = calculate_full_path_cost_final(problem_instance, new_gene)
    starting_gene_infeas, starting_gene_cost, starting_gene_infeas_nodes = calculate_full_path_cost_final(problem_instance, starting_gene)
    
    # Delta è una tupla (cambio_infeasibility, cambio_costo, infeasible_nodes)
    delta = (new_gene_infeas - starting_gene_infeas, new_gene_cost - starting_gene_cost, new_gene_infeas_nodes)

    return new_path, delta

In [51]:
def hill_climbing(problem_instance, initial_solution, n_iterations=1000):
    """
    Pure Hill Climbing (First Choice).
    Accetta solo mosse migliorative.
    """
    
    # 1. Inizializzazione
    current_solution = initial_solution.copy()
    
    # Calcolo costo iniziale COMPLETO una volta sola
    curr_infeas, curr_cost, _ = calculate_full_path_cost_final(problem_instance, current_solution)
    
    # Baseline per le mutazioni (calcolata una volta)
    path_list = neighborhood_greedy_strategy_dijistra(problem_instance)
    
    # Loop di ottimizzazione
    for iteration in range(n_iterations):
        
        # 2. Genera un vicino e ottieni la DELTA
        # new_path: il nuovo percorso proposto
        # delta_tuple: (diff_infeasibility, diff_cost, ...)
        neighbor_solution, delta_tuple = mutation_neighbor_of_next(current_solution, problem_instance, path_list)
        
        delta_infeas = delta_tuple[0]
        delta_cost = delta_tuple[1]
        
        # 3. Logica PURE HILL CLIMBING
        accept = False
        
        # Caso A: L'infeasibility diminuisce (MOLTO BENE) -> Accetta sempre
        if delta_infeas < 0:
            accept = True
            
        # Caso B: L'infeasibility aumenta (MALE) -> Rifiuta sempre
        elif delta_infeas > 0:
            accept = False
            
        # Caso C: L'infeasibility è invariata (uguale a prima)
        else: # delta_infeas == 0
            # Accetta solo se il costo diminuisce (o è uguale, per muoversi su plateau)
            if delta_cost <= 0:
                accept = True
            else:
                accept = False
        
        # 4. Aggiornamento (Solo se accettato)
        if accept:
            current_solution = neighbor_solution
            curr_infeas += delta_infeas
            curr_cost += delta_cost
            
            # Opzionale: stampa progressi se migliora
            # if delta_cost < 0:
            #     print(f"Iter {iteration}: Cost updated to {curr_cost:.2f}")

    return current_solution, curr_cost

In [55]:
P=Problem(1_000, density=0.2, alpha=1, beta=1)

In [58]:
y=neighborhood_greedy_strategy_dijistra(P)
y=choice_a_path(y,seed=random.randint(0,1000))
print(calculate_full_path_cost_final(P,y))

new_path, new_cost = hill_climbing(P,y, n_iterations=100)
print(new_path)
print(new_cost)
print(P.baseline())

(0, np.float64(195299.16560087368), [])


KeyboardInterrupt: 